In [1]:
import pandas as pd 
from census import Census
import gmaps

# Import API key
from google_api.api_key import apiKey

In [2]:
ev_data = ('resources/alt_fuel_stations_cleaned.csv')

In [3]:
ev_data = pd.read_csv(ev_data)
ev_data.head(1)

,Fuel Type Code,Station Name,Street Address,Intersection Directions,City,State,ZIP,Station Phone,Status Code,Groups With Access Code,...,ID,Updated At,Owner Type Code,Open Date,EV Connector Types,Country,Groups With Access Code (French),Access Code,Facility Type,EV Pricing
0,ELEC,Los Angeles Convention Center,1201 S Figueroa St,West hall and South hall,LOS ANGELES,CA,90015,213-741-1151,E,Public,...,1523,2020-11-09 16:53:32 UTC,P,1995-08-30,J1772,US,Public,public,PARKING_GARAGE,NaN


In [4]:
ev_data_clean = ev_data.drop(columns=['Street Address', 'Intersection Directions', 'Station Phone', 'Status Code',
                                     'Groups With Access Code', 'ID', 'Updated At', 'Owner Type Code', 'Open Date',
                                     'Groups With Access Code (French)', 'EV Network', 'EV Network Web'])
ev_data_clean.head(1)

,Fuel Type Code,Station Name,City,State,ZIP,Access Days Time,EV Level2 EVSE Num,Geocode Status,Latitude,Longitude,Date Last Confirmed,EV Connector Types,Country,Access Code,Facility Type,EV Pricing
0,ELEC,Los Angeles Convention Center,LOS ANGELES,CA,90015,24 hours daily; pay lot,12.0,GPS,34.040539,-118.271387,2020-11-09,J1772,US,public,PARKING_GARAGE,NaN


In [5]:
ev_data_clean["Unique City"] = ev_data_clean["City"] + ", " + ev_data_clean["State"]
ev_data_clean.head(1)

,Fuel Type Code,Station Name,City,State,ZIP,Access Days Time,EV Level2 EVSE Num,Geocode Status,Latitude,Longitude,Date Last Confirmed,EV Connector Types,Country,Access Code,Facility Type,EV Pricing,Unique City
0,ELEC,Los Angeles Convention Center,LOS ANGELES,CA,90015,24 hours daily; pay lot,12.0,GPS,34.040539,-118.271387,2020-11-09,J1772,US,public,PARKING_GARAGE,NaN,"LOS ANGELES, CA"


In [6]:
ev_state = ev_data_clean.groupby(['Unique City'])
ev_state_charges = ev_state.count()

In [7]:
ev_rename = ev_state_charges.rename(columns = {'Station Name': 'No. Charging Stations'})
ev_rename.head(3)

,Fuel Type Code,No. Charging Stations,City,State,ZIP,Access Days Time,EV Level2 EVSE Num,Geocode Status,Latitude,Longitude,Date Last Confirmed,EV Connector Types,Country,Access Code,Facility Type,EV Pricing
Unique City,,,,,,,,,,,,,,,,
"ABBOTSFORD, WI",1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1
"ABERDEEN, MD",6,6,6,6,6,6,5,6,6,6,6,6,6,6,1,4
"ABERDEEN, NJ",1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1


In [8]:
stations_count = ev_rename['No. Charging Stations']
stations_count

Unique City
ABBOTSFORD, WI         1
ABERDEEN, MD           6
ABERDEEN, NJ           1
ABERDEEN, SD           3
ABERDEEN, WA           1
                      ..
ZEPHYRHILLS, FL        5
ZILLAH, WA             1
ZION CROSSROADS, VA    1
ZIONSVILLE, IN         2
ZUMBROTA, MN           1
Name: No. Charging Stations, Length: 5700, dtype: int64

In [9]:
cities_data = ('resources/uscities.csv')

In [10]:
cities_data = pd.read_csv(cities_data)
cities_data.head(1)

,city,city_ascii,state_id,state_name,county_fips,county_name,lat,lng,population,density,source,military,incorporated,timezone,ranking,zips,id
0,New York,New York,NY,New York,36061,New York,40.6943,-73.9249,18713220,10715.0,polygon,False,True,America/New_York,1,11229 11226 11225 11224 11222 11221 11220 1138...,1840034016


In [11]:
cities_data_clean = cities_data.drop(columns=['city_ascii', 'state_name', 'county_fips', 'county_name',
                                     'source', 'military', 'incorporated', 'ranking'])
cities_data_clean.head(2)

,city,state_id,lat,lng,population,density,timezone,zips,id
0,New York,NY,40.6943,-73.9249,18713220,10715.0,America/New_York,11229 11226 11225 11224 11222 11221 11220 1138...,1840034016
1,Los Angeles,CA,34.1139,-118.4068,12750807,3276.0,America/Los_Angeles,90291 90293 90292 91316 91311 90037 90031 9000...,1840020491


In [12]:
cities_titles = cities_data_clean.rename(columns={
    'city': 'City',
    'state_id': 'State',
    'lat': 'Lat',
    'lng': 'Lng',
    'population': 'Population',
    'density': 'Density', 
    'timezone': 'Time Zone',
    'zips': 'ZIP',
    'id': 'ID'
})

cities_titles.head(2)

,City,State,Lat,Lng,Population,Density,Time Zone,ZIP,ID
0,New York,NY,40.6943,-73.9249,18713220,10715.0,America/New_York,11229 11226 11225 11224 11222 11221 11220 1138...,1840034016
1,Los Angeles,CA,34.1139,-118.4068,12750807,3276.0,America/Los_Angeles,90291 90293 90292 91316 91311 90037 90031 9000...,1840020491


In [13]:
cities_titles['City'] = cities_titles['City'].str.upper()
cities_titles.head(2)

,City,State,Lat,Lng,Population,Density,Time Zone,ZIP,ID
0,NEW YORK,NY,40.6943,-73.9249,18713220,10715.0,America/New_York,11229 11226 11225 11224 11222 11221 11220 1138...,1840034016
1,LOS ANGELES,CA,34.1139,-118.4068,12750807,3276.0,America/Los_Angeles,90291 90293 90292 91316 91311 90037 90031 9000...,1840020491


In [14]:
cities_titles["Unique City"] = cities_titles["City"] + ", " + cities_titles["State"]
cities_titles.head(2)

,City,State,Lat,Lng,Population,Density,Time Zone,ZIP,ID,Unique City
0,NEW YORK,NY,40.6943,-73.9249,18713220,10715.0,America/New_York,11229 11226 11225 11224 11222 11221 11220 1138...,1840034016,"NEW YORK, NY"
1,LOS ANGELES,CA,34.1139,-118.4068,12750807,3276.0,America/Los_Angeles,90291 90293 90292 91316 91311 90037 90031 9000...,1840020491,"LOS ANGELES, CA"


In [15]:
cities_count = cities_titles.groupby(['Unique City'])
cities_count.count()

,City,State,Lat,Lng,Population,Density,Time Zone,ZIP,ID
Unique City,,,,,,,,,
"AARONSBURG, PA",1,1,1,1,1,1,1,1,1
"ABANDA, AL",1,1,1,1,1,1,1,1,1
"ABBEVILLE, AL",1,1,1,1,1,1,1,1,1
"ABBEVILLE, GA",1,1,1,1,1,1,1,1,1
"ABBEVILLE, LA",1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...
"ZUMBROTA, MN",1,1,1,1,1,1,1,1,1
"ZUNI PUEBLO, NM",1,1,1,1,1,1,1,1,1
"ZURICH, KS",1,1,1,1,1,1,1,1,1


In [16]:
full_cities_ev = pd.merge(stations_count, cities_titles, on='Unique City', how='inner')
full_cities_ev.head(3)

,Unique City,No. Charging Stations,City,State,Lat,Lng,Population,Density,Time Zone,ZIP,ID
0,"ABBOTSFORD, WI",1,ABBOTSFORD,WI,44.9432,-90.3169,3833,285.0,America/Chicago,54405 54421,1840002168
1,"ABERDEEN, MD",6,ABERDEEN,MD,39.5146,-76.1730,16019,954.0,America/New_York,21001,1840005666
2,"ABERDEEN, SD",3,ABERDEEN,SD,45.4646,-98.4680,28134,680.0,America/Chicago,57401,1840002032


In [17]:
full_cities_ev.head()

,Unique City,No. Charging Stations,City,State,Lat,Lng,Population,Density,Time Zone,ZIP,ID
0,"ABBOTSFORD, WI",1,ABBOTSFORD,WI,44.9432,-90.3169,3833,285.0,America/Chicago,54405 54421,1840002168
1,"ABERDEEN, MD",6,ABERDEEN,MD,39.5146,-76.1730,16019,954.0,America/New_York,21001,1840005666
2,"ABERDEEN, SD",3,ABERDEEN,SD,45.4646,-98.4680,28134,680.0,America/Chicago,57401,1840002032
3,"ABERDEEN, WA",1,ABERDEEN,WA,46.9757,-123.8094,29609,595.0,America/Los_Angeles,98520,1840018446
4,"ABILENE, TX",3,ABILENE,TX,32.4543,-99.7384,116412,446.0,America/Chicago,79605 79699 79603 79601 79607 79606 79602 7960...,1840019476


In [18]:
heat_map = full_cities_ev[['Unique City','No. Charging Stations']].copy()
heat_map.to_csv('charging stations.csv')
heat_map.head()

,Unique City,No. Charging Stations
0,"ABBOTSFORD, WI",1
1,"ABERDEEN, MD",6
2,"ABERDEEN, SD",3
3,"ABERDEEN, WA",1
4,"ABILENE, TX",3


In [19]:
heat_map = full_cities_ev[['Unique City','No. Charging Stations','Lat', 'Lng']].copy()
heat_map.head()

,Unique City,No. Charging Stations,Lat,Lng
0,"ABBOTSFORD, WI",1,44.9432,-90.3169
1,"ABERDEEN, MD",6,39.5146,-76.1730
2,"ABERDEEN, SD",3,45.4646,-98.4680
3,"ABERDEEN, WA",1,46.9757,-123.8094
4,"ABILENE, TX",3,32.4543,-99.7384


In [20]:
gmaps.configure(api_key=apiKey)

In [25]:
location = heat_map[['Lat', 'Lng']]
weight = heat_map['No. Charging Stations']
chargings = gmaps.figure()
heat_layer = gmaps.heatmap_layer(location, weights=weight, dissipating=False, max_intensity=300, point_radius=1.5)
chargings.add_layer(heat_layer)
chargings

Figure()